In [1]:
import torch
import numpy as np
import data_loader
from torchvision import models
import torch.nn as nn
import torch.optim as optim

https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

In [2]:
BATCH_SIZE = 2
N_EPOCHS = 1
LR = 0.05
NUM_CLASSES = 14

Load the image data

https://pytorch.org/hub/pytorch_vision_resnet/

All pre-trained models expect input images normalized in the same way, i.e. mini-batches of 3-channel RGB images of shape (3 x H x W), where H and W are expected to be at least 224. The images have to be loaded in to a range of [0, 1] and then normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225].

In [31]:
image_loader_train, image_loader_validation = data_loader.load_data(BATCH_SIZE)

dataiter = iter(image_loader_train)
#print(len(image_loader_train))
test_mini_batch = next(dataiter)

# for batch_number, sample in enumerate(image_loader_train):
#     print(batch_number, sample["image"].shape, sample["target_labels"].shape)
    
#     if batch_number == 4:
#         break

#print(mini_batch["image"].shape)
#print(mini_batch["target_labels"].shape)

In [7]:
resnet = models.resnet34(weights='DEFAULT')

print("test batch: ", test_mini_batch["image"].shape, test_mini_batch["target_labels"].shape)

with torch.no_grad():
    output = resnet(test_mini_batch["image"])
    
print(output.shape)

test batch:  torch.Size([2, 3, 224, 224]) torch.Size([2, 14])
torch.Size([2, 1000])


Implement early stopping for regularization
(mun assignment kakkosesta)

In [4]:
class EarlyStopper:
    def __init__(self, patience=1):
        self.patience = patience
        self.wait = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                print("Early stopping due to improvement halt")
                return True
        return False

In [5]:
class MultilabelClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = models.resnet34(weights='DEFAULT')
        self.resnet.fc = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=self.resnet.fc.in_features, out_features=NUM_CLASSES)
            )
        self.model = self.resnet
        self.sigm = nn.Sigmoid()

    def forward(self, x):
        x = self.model(x)
        x = self.sigm(x)
        return x

In [6]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [7]:
model = MultilabelClassifier()
model.to(device)
optimizer = torch.optim.Adagrad(model.parameters(), lr=LR)
loss_function = nn.BCELoss()

In [18]:
def output_to_prediction(outputs, threshold=0.5):
    predictions = []
    for output in outputs:
        prediction = [np.float32(1.0) if i>0.5 else np.float32(0.0) for i in output]
        predictions.append(prediction)
    return torch.tensor(predictions).to(device)

In [19]:
#checks if all of the 14 labels are predicted correctly for one image (return True/False)
def prediction_fully_correct(prediction, target_label):
    return torch.equal(prediction, target_label)

In [28]:
#checks how many of the labels are predicted correctly for one image (returns int between 0 and 14)
def correct_labels_in_prediction(prediction, target_labels):
    return (prediction == target_labels).sum().item()

In [32]:
print("test batch: ", test_mini_batch["image"].shape, test_mini_batch["target_labels"].shape)
test_target = test_mini_batch["target_labels"].to(device)

with torch.no_grad():
    outputs = model(test_mini_batch["image"].to(device))
    
print(test_mini_batch["target_labels"])

print(outputs)

prediction = output_to_prediction(outputs)

print(prediction)

correct_prediction = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=np.float32)

print(correct_labels_in_prediction(prediction[0], test_target[0]))

test batch:  torch.Size([2, 3, 224, 224]) torch.Size([2, 14])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[0.0014, 0.0055, 0.0092, 0.0595, 0.0224, 0.1355, 0.0244, 0.1279, 0.0462,
         0.2953, 0.1520, 0.0076, 0.0042, 0.0330],
        [0.0047, 0.0213, 0.0132, 0.0581, 0.0377, 0.1930, 0.0382, 0.1787, 0.0091,
         0.3825, 0.2138, 0.0032, 0.0086, 0.0264]], device='cuda:0')
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
       device='cuda:0')
True
True
13


Train the model and validate

In [33]:
early_stopper = EarlyStopper(patience=3)
model.train()
for epoch in range(N_EPOCHS):
    train_loss = 0
    train_correct = 0
    total = 0
    total_correct_labels = 0
    for batch_number, data in enumerate(image_loader_train):
        images, target_labels = data['image'].to(device), data['target_labels'].to(device)

        outputs = model(images)
        #print("outputs: ", outputs.is_cuda)
        loss = loss_function(outputs, target_labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        total += target_labels.size(0)
        predictions = output_to_prediction(outputs)
        #print("prediction: ", predictions.is_cuda)
        for i, prediction in enumerate(predictions):
            if prediction_fully_correct(prediction, target_labels[i]):
                train_correct += 1
            total_correct_labels += correct_labels_in_prediction(prediction, target_labels[i])

        print('Training: Epoch %d - Batch %d/%d: Loss: %.4f | Train fully correct: %.3f%% (%d/%d) - Train labels correct: %.3f%% (%d/%d)' % 
              (epoch, batch_number + 1, len(image_loader_train), train_loss / (batch_number + 1), 
               100. * train_correct / total, train_correct, total, 100. * total_correct_labels / (total*14), total_correct_labels, (total*14)))

True
True
True
True
Training: Epoch 0 - Batch 1/8000: Loss: 0.3292 | Train fully correct: 0.000% (0/2) - Train labels correct: 82.143% (23/28)
True
True
True
True
Training: Epoch 0 - Batch 2/8000: Loss: 0.3094 | Train fully correct: 0.000% (0/4) - Train labels correct: 83.929% (47/56)
True
True
True
True
Training: Epoch 0 - Batch 3/8000: Loss: 0.2928 | Train fully correct: 16.667% (1/6) - Train labels correct: 88.095% (74/84)
True
True
True
True
Training: Epoch 0 - Batch 4/8000: Loss: 0.2653 | Train fully correct: 25.000% (2/8) - Train labels correct: 90.179% (101/112)
True
True
True
True
Training: Epoch 0 - Batch 5/8000: Loss: 0.2698 | Train fully correct: 20.000% (2/10) - Train labels correct: 89.286% (125/140)
True
True
True
True
Training: Epoch 0 - Batch 6/8000: Loss: 0.2546 | Train fully correct: 25.000% (3/12) - Train labels correct: 89.881% (151/168)
True
True
True
True
Training: Epoch 0 - Batch 7/8000: Loss: 0.2294 | Train fully correct: 35.714% (5/14) - Train labels correct: 9

KeyboardInterrupt: 